In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
# import [internal library]
import matplotlib.pyplot as plt
import datetime
import re

from darts import TimeSeries
#import datetime as dt

# Setup

In [ ]:
# Redacted
# Use internal libraries for database connections

# Data

## Average CPC and ROAS query

In [ ]:
# Redacted
# Query to pull average CPC and ROAS by producct taxonomy
cpc_query = ""
cpc_data = pd.DataFrame()

## Bid Floors

In [ ]:
# Redacted
bid_floor_query = ""
bid_floor_data = pd.DataFrame()

## Test Categories
Using level 1 and level 2 taxonomies

In [ ]:
# YTD Metrics
year_start = datetime.datetime(2024, 1, 1).date()
cpc_ytd = cpc_data[cpc_data["month"].dt.date >= year_start]

# Filter categories by ROAS
roas = (cpc_ytd
 .groupby(["taxonomy_l1", "taxonomy_l2"])
 [["sales", "spend"]]
 .sum()
 .reset_index()
 .assign(roas=lambda x: x.sales / x.spend))

In [ ]:
# Test categories: choose only categories with high ROAS (above roas_lim)
roas_lim = 4 # very conservative
roas_filtered = roas.query(f'roas > {roas_lim}')
test_categories = roas_filtered[["taxonomy_l1", "taxonomy_l2"]].reset_index(drop=True)

In [ ]:
# Filter data to test categories
cpc_data_filtered = cpc_data.merge(test_categories)

In [ ]:
# Join to bid floors, then aggregate
cpc_data_final = (cpc_data_filtered
 .merge(bid_floor_data)
 .groupby(["month", "taxonomy_l1", "taxonomy_l2"])
 .agg({
     "clicks": "sum",
     "spend": "sum",
     "min_cpc": "mean"
 })
 .reset_index()
 .assign(avg_cpc = lambda x: x.spend / x.clicks)
 )
cpc_data_final["month"] = cpc_data_final["month"].dt.strftime('%Y-%m-%d')

# Create Time Series Forecasts

In [142]:
from darts.models import StatsForecastAutoARIMA
from darts.metrics import mape

## Generate forecast function

In [143]:
# Function
def generate_forecast(test_index):
    
    try:
        idx = test_index
        
        # Create data frame for single time series
        category = test_categories[test_categories.index == idx].reset_index()
        ts_df = cpc_data_final.merge(category)
        ts_df = ts_df.drop(ts_df.tail(1).index) # Drop current month
        tax_name = category["taxonomy_l1"][0] + ' > ' + category["taxonomy_l2"][0]
    
    except:
        print(f"Failed to create forecast DataFrame for category index {test_index}")
    
    try:
        # Create monthly time series
        ts = TimeSeries.from_dataframe(ts_df, time_col='month', freq='MS', value_cols=['avg_cpc'])
        
        # Training data is first two years
        train, val = ts.split_before(pd.Timestamp('2023-08-01'))
        
        # Fit model to training data and store MAPE
        arima_train = StatsForecastAutoARIMA()
        arima_train.fit(train)
        forecast = arima_train.predict(len(val), num_samples=1000)
        validation_mape = mape(forecast, val) / 100 
        
        # Fit full model
        arima = StatsForecastAutoARIMA()
        arima.fit(ts)
        forecast = arima.predict(3, num_samples=1000)
        
        # Extract forecast estimates and confidence intervals
        hilo = forecast.quantiles_df((0.05, 0.5, 0.95)) # 90% CI
        hilo["taxonomy_l1"] = category["taxonomy_l1"][0]
        hilo["taxonomy_l2"] = category["taxonomy_l2"][0]
        hilo["bid_floor"] = ts_df.sort_values("month").tail(1)["min_cpc"].reset_index(drop=True)[0]
        hilo["mape"] = validation_mape

        # Aggregate values over next quarter   
        hilo_agg = (hilo
            .groupby(["taxonomy_l1", "taxonomy_l2", "bid_floor", "mape"])
            .agg({"avg_cpc_0.05": "min", 
                "avg_cpc_0.5": "median", 
                "avg_cpc_0.95":"max"})
            .reset_index()
            .rename(columns={
                "avg_cpc_0.05": "lower_bound", 
                "avg_cpc_0.5": "est", 
                "avg_cpc_0.95": "upper_bound"
            })
        )
        
        return hilo_agg
    
    except:
        print(f"Failed to create forecast for {tax_name}")

## Loop: attempt to generate forecasts for all categories

In [144]:
df = pd.DataFrame()

for i in range(0, len(test_categories.index) - 1):
    fcast = generate_forecast(i)
    df = pd.concat([df, fcast])

ERROR:main_logger:ValueError: Train series only contains 1 elements but StatsForecastAutoARIMA() model requires at least 10 entries


Failed to create forecast for car electronics & gps > car video & dvd


ERROR:darts.timeseries:ValueError: Timestamp must be between 2023-11-01 00:00:00 and 2024-01-01 00:00:00
ERROR:main_logger:ValueError: Train series only contains 5 elements but StatsForecastAutoARIMA() model requires at least 10 entries


Failed to create forecast for cell phones > lively phones
Failed to create forecast for cell phones > samsung galaxy


ERROR:darts.timeseries:ValueError: Timestamp must be between 2022-05-01 00:00:00 and 2023-07-01 00:00:00


...

Failed to create forecast for smart home > smart home hubs


ERROR:main_logger:ValueError: Train series only contains 1 elements but StatsForecastAutoARIMA() model requires at least 10 entries


Failed to create forecast for video games > all video games


ERROR:main_logger:ValueError: Train series only contains 0 elements but StatsForecastAutoARIMA() model requires at least 10 entries


Failed to create forecast for video games > virtual reality


## Process Final Data

In [ ]:
# Remove high volatility categories we can't forecast well
mape_limit = 0.3

# Create final DataFrame
df_final = (df
            # Negative lower bound isn't meaningful, so set to zero
            .assign(lower_bound = lambda x: np.where(x.lower_bound < 0, 0, x.lower_bound))
            .query(f'mape < {mape_limit}')
            .reset_index(drop=True)
            )

n_categories_final = len(df_final.index)
n_categories_start = len(roas.index)
forecasted_category_pct = round(n_categories_final / n_categories_start * 100)
print(f'Successfully generated quarterly forecasts for {n_categories_final} of {n_categories_start} categories ({forecasted_category_pct}%) with ROAS > {roas_lim} and MAPE < {round(mape_limit * 100)}%!')

Successfully generated quarterly forecasts for 61 of 108 categories (56%) with ROAS > 4 and MAPE < 30%!


# Write to file

In [147]:
df_final.to_csv(f'output/taxonomy_l2_forecasts_bid_floors_{retailer_id}.csv', index=False)